## Автоматический поиск решения головоломки "Найди 36 видов рыб"
### Вариант с использованием **NumPy**

Входные данные:
- файл `fish_field.csv` с игровым полем головоломки
- файл `fish_biglist.csv` со списком разных названий видов рыб (по первым попавшимся данным из Интернета)

Выходные данные:
- список объектов класса FishWord, содержащий все найденные варианты названий рыб

In [1]:
import numpy as np
import enum
import itertools


class WordDir(enum.IntEnum):
    '''
    Расположение (направление) слова на поле головоломки
    '''
    dirUpDiag = 0     # по диагонали вправо-вверх
    dirHor = 1        # по горизонтали
    dirDownDiag = 2   # по диагонали вправо-вниз
    dirVert = 3       # по вертикали
    def __str__(self):
        return ('диаг. вправо-вверх', 'горизонтали', 'диаг. вправо-вниз', 'вертикали')[self.value]

class FishWord():
    '''
    Класс для объектов с описанием положения вида рыбы на поле головоломки
    '''
    def __init__(self, fishname: str, direction: WordDir, row: int, col: int) -> None:
        self.word_fishname = fishname
        self.word_row = row + 1
        self.word_col = col + 1
        self.word_direction = direction

    def __str__(self) -> str:
        return f'[{self.word_row}:{self.word_col}] по {self.word_direction} - {self.word_fishname}'
        

def check_letters(some_letters: np.ndarray, direction: WordDir, row: int, col: int):
    '''
    Функция проверки всех возможных слов из букв, перечисленных в массиве word_letters
    (начиная с первых двух букв с увеличением на 1 символ), на соответствие реальным
    названиям рыб из глобального списка fishes и, в случае успеха, добавления
    этих вариантов в глобальный список объектов fishwords
    '''
    if len(some_letters) < 2:
        return
    global fishwords
    global fishes
    new_word = some_letters[0].replace('Ё', 'Е')
    for ind in range(1, len(some_letters)):
        new_word += some_letters[ind].replace('Ё', 'Е')
        if new_word in fishes:
            fishwords.append(FishWord(new_word, direction, row, col))
    return


# Список для сбора данных о найденных словах
fishwords = []

# Формируем set fishes c перечнем разных названий видов рыб из файла fish_biglist.csv
fishes = {fishname for textline in np.genfromtxt(fname="fish_biglist.csv", dtype=str, delimiter=";", skip_header=1, encoding='UTF-8') \
        for fishname in str(textline).upper().replace("(","").replace(")","").replace("-"," ").replace("Ё","Е").split()}

# Формируем NumPy массив letters с полем головоломки из файла fish_field.csv
letters = np.genfromtxt(fname="fish_field.csv", dtype=str, delimiter=";", skip_header=1, encoding='UTF-8')
    
# Количество строк и столбцов
len_rr = len(letters)
len_cc = len(letters[0])

# Перебираем все строки (rr) и столбцы в них (cc)
for rr, cc in itertools.product(range(len_rr), range(len_cc)):
    # Горизонталь
    check_letters(letters[rr, cc:len_cc], WordDir.dirHor, rr, cc)
    # Вертикаль
    check_letters(letters[rr:len_rr, cc], WordDir.dirVert, rr, cc)
    # Диагональ вправо-вверх 
    max_len = min(rr + 1, len_cc - cc)
    if max_len > 1:
        check_letters(letters[range(rr, rr - max_len, -1), range(cc, cc + max_len)], WordDir.dirUpDiag, rr, cc)
    # Диагональ вправо-ввниз 
    max_len = min(len_rr - rr, len_cc - cc)
    if max_len > 1:
        check_letters(letters[range(rr, rr + max_len), range(cc, cc + max_len)], WordDir.dirDownDiag, rr, cc)
                
# Список объектов готов. Можно его использовать дальше.
# Например, отсортировать в другом порядке и вывести принтом

fishwords.sort(key=lambda x: x.word_row * 100 + x.word_col)
for cnt, fishword in enumerate(fishwords):
    print(f'{cnt + 1}) {fishword}')


1) [1:1] по горизонтали - ОСЕТР
2) [1:1] по вертикали - ОКУНЬ
3) [1:10] по горизонтали - ЕЛЕЦ
4) [1:11] по диаг. вправо-вниз - ЛИНЬ
5) [2:1] по горизонтали - КЕФАЛЬ
6) [2:1] по диаг. вправо-вниз - КАМБАЛА
7) [3:5] по горизонтали - ТУНЕЦ
8) [3:5] по диаг. вправо-вниз - ТРЕСКА
9) [3:12] по вертикали - СУДАК
10) [4:2] по диаг. вправо-вниз - НАЛИМ
11) [5:3] по вертикали - АКУЛА
12) [5:9] по диаг. вправо-вниз - КАРП
13) [5:11] по вертикали - ФОРЕЛЬ
14) [5:14] по вертикали - ТИЛАПИЯ
15) [6:1] по диаг. вправо-вниз - ГОЛЕЦ
16) [6:2] по вертикали - МОЛОТ
17) [6:3] по диаг. вправо-вниз - КОРЮШКА
18) [6:6] по диаг. вправо-вниз - ПЕСКАРЬ
19) [6:13] по вертикали - СЕМГА
20) [7:8] по диаг. вправо-вверх - КЕТА
21) [7:10] по вертикали - САЗАН
22) [8:4] по вертикали - БЕЛУГА
23) [8:9] по диаг. вправо-вниз - ЯЗЬ
24) [9:7] по горизонтали - ХЕК
25) [9:9] по диаг. вправо-вверх - КАРАСЬ
26) [10:1] по вертикали - ПИКША
27) [11:2] по вертикали - СОМ
28) [11:2] по диаг. вправо-вверх - СТЕРЛЯДЬ
29) [11:3] по го

### Пример использования списка - формирование констаны для скрипта js

In [2]:
clip_text = "const answers = ["
fishwords.sort(key=lambda x: x.word_fishname)
for cnt, fishword in enumerate(fishwords):
    clip_text += "{" + f'"value":"{fishword.word_fishname.capitalize()}","letters":['
    first_letter = (fishword.word_row - 1) * 14 + fishword.word_col - 1
    rr_delta = [-1, 0, 1, 1][fishword.word_direction]
    cc_delta = [1, 1, 1, 0][fishword.word_direction]
    clip_text += ",".join(
        [
            str(first_letter + letter_pos * rr_delta * 14 + letter_pos * cc_delta)
            for letter_pos in range(len(fishword.word_fishname))
        ]
    )
    clip_text += "]},"
clip_text += "];"
print(clip_text)

const answers = [{"value":"Акула","letters":[58,72,86,100,114]},{"value":"Белуга","letters":[101,115,129,143,157,171]},{"value":"Голавль","letters":[159,160,161,162,163,164,165]},{"value":"Голец","letters":[70,85,100,115,130]},{"value":"Елец","letters":[9,10,11,12]},{"value":"Зубатка","letters":[142,143,144,145,146,147,148]},{"value":"Камбала","letters":[14,29,44,59,74,89,104]},{"value":"Карась","letters":[120,107,94,81,68,55]},{"value":"Карп","letters":[64,79,94,109]},{"value":"Кета","letters":[91,78,65,52]},{"value":"Кефаль","letters":[14,15,16,17,18,19]},{"value":"Корюшка","letters":[72,87,102,117,132,147,162]},{"value":"Лещ","letters":[161,176,191]},{"value":"Линь","letters":[10,25,40,55]},{"value":"Лосось","letters":[185,186,187,188,189,190]},{"value":"Молот","letters":[71,85,99,113,127]},{"value":"Налим","letters":[43,58,73,88,103]},{"value":"Окунь","letters":[0,14,28,42,56]},{"value":"Осетр","letters":[0,1,2,3,4]},{"value":"Палтус","letters":[170,171,172,173,174,175]},{"value":"